##### Cargue de librerias

In [1]:
from pycaret.clustering import *
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import plotly.express as px
import plotly.figure_factory as ff
import plotly.subplots as sp
import plotly.graph_objs as go

import warnings
warnings.filterwarnings("ignore")

##### Conexion al DWH de Dispapeles y carga de bds adicionales

In [2]:
def df_clustering(consulta_SQL):
    ## Conexion al DWH
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()
    
    #Cargue de la data desde el ERP de Dispapeles y se guarda en df
    cursor.execute(consulta_SQL)
    rows = cursor.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

    return df

##### Funcion EDA

In [23]:
def histograma(bd):
    hist_data = [bd["Escala R vta"], bd["Escala F vta"], bd["Escala M vta"], bd["Escala R rdo"], bd["Escala F rdo"], bd["Escala M rdo"]]
    group_labels = ["Escala R vta", "Escala F vta", "Escala M vta", "Escala R rdo", "Escala F rdo", "Escala M rdo"]
    fig = sp.make_subplots(rows=1, cols=3)
    
    for j in range(2):
        for i in range(3):
            fig.add_trace(go.Histogram(x=hist_data[i*j], showlegend=False, histfunc="avg"), row=1+j, col=i+1)
    fig.update_layout(
        title="Histogramas de Escala R, Escala F y Escala M",
        xaxis=dict(title="Valor"),
        yaxis=dict(title="Frecuencia"),
        showlegend=False
    )
    fig.show()

def matrix(bd, col):
    corr_matrix = bd[col].corr()
    fig = px.imshow(corr_matrix, text_auto=True)
    fig.show()
    
def EDA(bd):
    bd = bd.reset_index()
    print(f"Esta base de datos tiene {len(bd)} registros,")
    print(f"hace parte del distrito {bd['Codigo distrito'][0]}")
    print(f"y el tipo de zona es {bd['Nombre tipo zona'][0]}")
    
    print(" ")
    print("Describe de la base de datos")
    describe = bd.describe().applymap("{:,.0f}".format)
    print(describe[["Escala R vta", "Escala F vta", "Escala M vta", "Escala R rdo", "Escala F rdo", "Escala M rdo"]][1:])
    
    print(" ")
    
    # histograma(bd)
    matrix(bd, ["Escala R vta", "Escala F vta", "Escala M vta", "Escala R rdo", "Escala F rdo", "Escala M rdo"])

##### Funciones estandarizacion

In [4]:
# Crear una función para asignar etiquetas basadas en percentiles
def asignar_escala_R(valor, escalas_R= [24, 21, 15, 8, 0]):
    if valor >= escalas_R[0]:
        return 5
    elif valor >= escalas_R[1]:
        return 4
    elif valor >= escalas_R[2]:
        return 3
    elif valor >= escalas_R[3]:
        return 2
    else:
        return 1
    
def asignar_escala_F(valor, escalas_F= [24, 21, 15, 8, 0]):
    if valor >= escalas_F[0]:
        return 5
    elif valor >= escalas_F[1]:
        return 4
    elif valor >= escalas_F[2]:
        return 3
    elif valor >= escalas_F[3]:
        return 2
    else:
        return 1
    
def asignar_escala_M(valor):
    if valor <= escalas_M[1]:
        return 1
    elif valor <= escalas_M[2]:
        return 2
    elif valor <= escalas_M[3]:
        return 3
    elif valor <= escalas_M[4]:
        return 4
    else:
        return 5
    

##### Carga de la base de datos

In [5]:
# Consulta SQL para descargar la informacion a usar
codigo_SQL_vta = "SELECT A.DSTCOD AS 'Codigo distrito', B.VNDZNA AS 'Codigo zona actual', CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC) AS 'Nit cliente-sucursal', A.VTAANO AS 'Ano', A.VTAMES AS 'Mes', (((2021 - MAX(A.VTAANO)) * 12 ) + (0 - MAX(A.VTAMES))) * -1 AS 'Recency', SUM(A.VTAVLRVTA) AS 'Monetary', COUNT( DISTINCT A.VTAMES) AS 'Frequency' FROM V_VTA_VTAHEC A LEFT JOIN	DIM_CLIENTES B ON A.PRMCOD = B.PRMCOD AND A.DSTCOD = B.DSTCOD AND A.VTANIT = B.CLTNIT AND A.VTASUC = B.CLTSUC WHERE A.PRMCOD = 1 AND A.DSTCOD NOT IN (30,35) AND A.VTAZNA < 96 AND A.VTAFCH BETWEEN '2021-01-01' AND '2022-12-31' GROUP BY A.DSTCOD, B.VNDZNA, CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC), A.VTAANO, A.VTAMES HAVING SUM(A.VTAVLRVTA) > 0"
codigo_SQL_rdo = "SELECT Distrito AS 'Codigo distrito', Zona AS 'Codigo zona actual', CONCAT(CONCAT(Cliente,'-'), Suc) AS 'Nit cliente-sucursal', YEAR(Fch_PagoRecaudo) AS 'Ano', MONTH(Fch_PagoRecaudo) AS 'Mes', (((2021 - MAX(YEAR(Fch_PagoRecaudo))) * 12 ) + (0 - MAX(MONTH(Fch_PagoRecaudo)))) * -1 AS 'Recency', SUM(SinVenc + [Venc.30] + [Venc.60] + [Venc.90] + [Venc.120] + [Venc++120]) 'Monetary', COUNT(DISTINCT MONTH(Fch_PagoRecaudo)) AS 'Frequency' FROM RECAUDOS WHERE Distrito NOT IN (30, 35) AND Fch_PagoRecaudo BETWEEN '2021-01-01' AND '2022-12-31' AND Empresa = 1 GROUP BY Distrito, Zona, CONCAT(CONCAT(Cliente,'-'), Suc), YEAR(Fch_PagoRecaudo), MONTH(Fch_PagoRecaudo) HAVING	SUM(SinVenc + [Venc.30] + [Venc.60] + [Venc.90] + [Venc.120] + [Venc++120]) > 0"

# Consulta adicional para establecer el tipo de zona
maestra_zonas = pd.read_csv('C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/Análisis clustering/Maestra_zonas.csv', delimiter= ";")

In [6]:
df_vta = df_clustering(codigo_SQL_vta)
df_rdo = df_clustering(codigo_SQL_rdo)
# df = df_clustering(codigo_SQL)

##### Tranformaciones

In [7]:
# Conversion de tipo de la columna Monetary
df_vta["Monetary"] = df_vta["Monetary"].astype(int)
df_rdo["Monetary"] = df_rdo["Monetary"].astype(np.int64)

# Join con la maestra de zonas para determinar el tipo de zona
df_vta2 = pd.merge(df_vta, maestra_zonas, on= ["Codigo distrito", "Codigo zona actual"])
df_rdo2 = pd.merge(df_rdo, maestra_zonas, on= ["Codigo distrito", "Codigo zona actual"])

# Filtro de la informacion para excluir zonas no relevantes
list_filter = ['Artes graficas', 'TIG', 'V Horeca', 'Institucional', 'V SyE',
                'V Industria', 'Mayorista']
df_vta = df_vta2[df_vta2["Nombre tipo zona"].isin(list_filter)]
df_rdo = df_rdo2[df_rdo2["Nombre tipo zona"].isin(list_filter)]

# DF final agregada por tipo de zona
df_vta3 = df_vta.groupby(["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal"]).agg({"Recency": np.max,
                                                                            "Monetary": np.sum,
                                                                            "Frequency": np.sum
                                                                            })
df_rdo3 = df_rdo.groupby(["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal"]).agg({"Recency": np.max,
                                                                            "Monetary": np.sum,
                                                                            "Frequency": np.sum
                                                                            })
df_vta3 = df_vta3.reset_index(col_level= 1)
df_rdo3 = df_rdo3.reset_index(col_level= 1)

df_3 = pd.merge(df_vta3, df_rdo3, on=["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal"], how="left")
df_3 = df_3.fillna(1)

In [11]:
df_3.columns

Index(['Codigo distrito', 'Nombre tipo zona', 'Nit cliente-sucursal',
       'Recency_x', 'Monetary_x', 'Frequency_x', 'Recency_y', 'Monetary_y',
       'Frequency_y'],
      dtype='object')

In [13]:
# Se añade una key de Distrito-Nombretipozona para hacer mas facil la identificacion cuando se usa todo el pais
df_3['Distrito-Nombretipozona'] = df_3['Codigo distrito'].astype(str) + '-' + df_3['Nombre tipo zona']

# Venta
escalas_M = np.percentile(df_3["Monetary_x"].values, [0, 20, 50, 80, 95, 100])
df_est = df_3
df_est["Escala R vta"] = df_3["Recency_x"].apply(asignar_escala_R, escalas_R= [24, 21, 15, 8, 0])
df_est["Escala F vta"] = df_3["Frequency_x"].apply(asignar_escala_F, escalas_F= [24, 21, 15, 8, 0])
df_est["Escala M vta"] = df_3["Monetary_x"].apply(asignar_escala_M)
df_est = df_est.drop(["Recency_x", "Frequency_x", "Monetary_x"], axis=1)

# Recaudo
escalas_M = np.percentile(df_3["Monetary_y"].values, [0, 20, 50, 80, 95, 100])
df_est["Escala R rdo"] = df_3["Recency_y"].apply(asignar_escala_R, escalas_R= [24, 21, 15, 8, 0])
df_est["Escala F rdo"] = df_3["Frequency_y"].apply(asignar_escala_F, escalas_F= [24, 21, 15, 8, 0])
df_est["Escala M rdo"] = df_3["Monetary_y"].apply(asignar_escala_M)
df_est = df_est.drop(["Recency_y", "Frequency_y", "Monetary_y"], axis=1)
df_est

,Codigo distrito,Nombre tipo zona,Nit cliente-sucursal,Distrito-Nombretipozona,Escala R vta,Escala F vta,Escala M vta,Escala R rdo,Escala F rdo,Escala M rdo
0,10,Artes graficas,1010197660-1,10-Artes graficas,2,1,2,1,1,1
1,10,Artes graficas,1010208977-1,10-Artes graficas,1,1,2,1,1,1
2,10,Artes graficas,1012348815-1,10-Artes graficas,1,1,1,1,1,1
3,10,Artes graficas,1013637504-1,10-Artes graficas,1,1,1,1,1,1
4,10,Artes graficas,1013642113-1,10-Artes graficas,5,1,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...
14383,90,Mayorista,93300278-1,90-Mayorista,3,1,2,1,1,1
14384,90,Mayorista,93370623-1,90-Mayorista,5,4,3,1,1,1
14385,90,Mayorista,93386132-1,90-Mayorista,4,3,5,4,2,5
14386,90,Mayorista,93414007-1,90-Mayorista,5,3,3,1,1,1


##### Creacion, ajuste y descarga de los modelos de clustering por cada distrito-tipo de zona

In [14]:
list_dtos = df_est["Distrito-Nombretipozona"].unique()
numeric_features = ["Escala R vta", "Escala F vta", "Escala M vta", "Escala R rdo", "Escala F rdo", "Escala M rdo"]
ignore_columns = ["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal", "Distrito-Nombretipozona"]

#### EDA

In [15]:
df_EDA = df_est[df_est["Distrito-Nombretipozona"] == '10-Artes graficas']

In [24]:
EDA(df_EDA)

Esta base de datos tiene 524 registros,
hace parte del distrito 10
y el tipo de zona es Artes graficas
 
Describe de la base de datos
     Escala R vta Escala F vta Escala M vta Escala R rdo Escala F rdo  \
mean            4            2            3            3            1   
std             1            1            1            2            0   
min             1            1            1            1            1   
25%             3            1            2            1            1   
50%             4            1            3            3            1   
75%             5            3            4            5            2   
max             5            5            5            5            2   

     Escala M rdo  
mean            3  
std             2  
min             1  
25%             1  
50%             3  
75%             4  
max             5  
 


In [ ]:
# list_dtos = ['10-Artes graficas', '10-Institucional', '10-Mayorista', '10-TIG', '10-V Horeca', '10-V Industria', '10-V SyE']
# list_dtos = ['10-Artes graficas']

##### Un modelo

In [29]:
# Variables iniciales
metrics = []
bd_predicted_all = pd.DataFrame()

# Fecha para guardar resultados
fecha = '30-04-23'

# Ciclo para realizar el modelo de clustering por cada tipo de zona del pais
for n, dto in enumerate(list_dtos):
    print(f"Procesando modelo {n+1} de {len(list_dtos)}. {(n+1) / (len(list_dtos)):.1%}")
    
    #BD filtrada
    df_dto = df_est[df_est["Distrito-Nombretipozona"] == dto]

    #Setup del modelo 
    s = setup(
                df_dto,
                verbose= False,
                preprocess= False,
                profile= False,
                ignore_features= ignore_columns,
                numeric_features= numeric_features
                )
    
    #Creacion y prediccion de modelo
    model_kmeans = create_model(
                                'kmeans',
                                num_clusters= 5,
                                verbose= False,
                                )
    pull()
    df_pred_km = predict_model(
                                model_kmeans,
                                data = df_dto
                                )
    
    # Reasignacion de los nombres de los clusters
    trans = df_pred_km.groupby(
                                ["Cluster"]).agg({
                                                "Escala R vta": np.mean, "Escala F vta": np.mean, "Escala M vta": np.mean,
                                                "Escala R rdo": np.mean, "Escala F rdo": np.mean, "Escala M rdo": np.mean,
                                            }).reset_index()
    trans["avg"] = (trans["Escala R vta"] + trans["Escala F vta"] + trans["Escala M vta"] + 
                    trans["Escala R rdo"] + trans["Escala F rdo"] + trans["Escala M rdo"]) / 6
    trans = trans.sort_values(by= ["avg"], ascending= False)
    trans["Letra cluster"] = ["A", "B", "C", "D", "E"]
    trans = trans[["Cluster", "Letra cluster"]]
    
    df_pred_km = pd.merge(df_pred_km, trans, on= ["Cluster"])
    df_pred_km = df_pred_km.drop_duplicates(subset=["Escala R vta", "Escala F vta", "Escala M vta",
                                                    "Escala R rdo", "Escala F rdo", "Escala M rdo"])
    
    # Descarga del modelo
    bd_predicted = pd.merge(df_dto, df_pred_km, on= ["Escala R vta", "Escala F vta", "Escala M vta",
                                                     "Escala R rdo", "Escala F rdo", "Escala M rdo"])

    bd_predicted_all = pd.concat([bd_predicted_all, bd_predicted], ignore_index=  True)

    #Append a la variable incial de metricas
    metrics.append({
                    "Distrito-zona": dto,
                    "Silhouette": pull().Silhouette[0],
                    "Calinski-Harabasz": pull()["Calinski-Harabasz"][0],
                    "Davies-Bouldin": pull()["Davies-Bouldin"][0]
                    })
    # save_model(model_kmeans, f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Modelos {fecha}/Modelo {dto} {fecha}")

# Transformacion de la variable de metricas
metrics = pd.DataFrame(metrics)
metrics["Tipo zona"] = metrics["Distrito-zona"].str[3:]

print("Proceso completo 100%")

Procesando modelo 1 de 67. 1.5%
Procesando modelo 2 de 67. 3.0%
Procesando modelo 3 de 67. 4.5%
Procesando modelo 4 de 67. 6.0%
Procesando modelo 5 de 67. 7.5%
Procesando modelo 6 de 67. 9.0%
Procesando modelo 7 de 67. 10.4%
Procesando modelo 8 de 67. 11.9%
Procesando modelo 9 de 67. 13.4%
Procesando modelo 10 de 67. 14.9%
Procesando modelo 11 de 67. 16.4%
Procesando modelo 12 de 67. 17.9%
Procesando modelo 13 de 67. 19.4%
Procesando modelo 14 de 67. 20.9%
Procesando modelo 15 de 67. 22.4%
Procesando modelo 16 de 67. 23.9%
Procesando modelo 17 de 67. 25.4%
Procesando modelo 18 de 67. 26.9%
Procesando modelo 19 de 67. 28.4%
Procesando modelo 20 de 67. 29.9%
Procesando modelo 21 de 67. 31.3%
Procesando modelo 22 de 67. 32.8%
Procesando modelo 23 de 67. 34.3%
Procesando modelo 24 de 67. 35.8%
Procesando modelo 25 de 67. 37.3%
Procesando modelo 26 de 67. 38.8%
Procesando modelo 27 de 67. 40.3%
Procesando modelo 28 de 67. 41.8%
Procesando modelo 29 de 67. 43.3%
Procesando modelo 30 de 67. 4

In [30]:
# Descarga de bd predicha y metricas calculadas
bd_predicted_all.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering vta + rdo {fecha}.csv",
                        encoding= 'utf-8', index= False, decimal= ",", sep= ";")
metrics.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Metricas vta + rdo {fecha}.csv",
                        encoding= 'utf-8', index= False, decimal= ",",  sep= ";")

In [ ]:
fig = px.scatter_3d(df_pred_km.sort_values(by= "Letra cluster"), x= "Escala R", y= "Escala F", z= "Escala M",
                    color= "Letra cluster")
fig.show()

In [31]:
plot_model(model_kmeans, plot = 'cluster')

In [32]:
plot_model(model_kmeans, plot = 'silhouette')

In [34]:
plot_model(model_kmeans, plot = 'distribution')

In [33]:
trans

,Cluster,Letra cluster
4,Cluster 4,A
0,Cluster 0,B
1,Cluster 1,C
3,Cluster 3,D
2,Cluster 2,E
